# Stacking Activity

In [1]:
import numpy as np
import pandas as pd

mydata = pd.read_csv("/Users/jaredmcmullen/Desktop/GSB-S545/data/abalonedata.csv")
mydata

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
...,...,...,...,...,...,...,...,...,...
4141,F,0.650,0.505,0.165,1.3570,0.5725,0.2810,0.430,11
4142,M,0.655,0.525,0.180,1.4020,0.6240,0.2935,0.365,13
4143,F,0.655,0.500,0.220,1.3590,0.6420,0.3255,0.405,13
4144,M,0.670,0.535,0.190,1.6690,0.7465,0.2935,0.508,11


In [2]:
# create the boolean variables
mydata =  pd.get_dummies(mydata) 
mydata

,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings,Sex_F,Sex_I,Sex_M
0,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,0,0,1
1,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,0,0,1
2,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,1,0,0
3,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,0,0,1
4,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
4141,0.650,0.505,0.165,1.3570,0.5725,0.2810,0.430,11,1,0,0
4142,0.655,0.525,0.180,1.4020,0.6240,0.2935,0.365,13,0,0,1
4143,0.655,0.500,0.220,1.3590,0.6420,0.3255,0.405,13,1,0,0
4144,0.670,0.535,0.190,1.6690,0.7465,0.2935,0.508,11,0,0,1


In [3]:
from sklearn.model_selection import train_test_split

# Remove Rings and convert to 2D Array
y = mydata.Rings.values
del mydata['Rings']
X = mydata.values.astype(float)

In [15]:
#Select all the models you want to test
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor

def get_stacking():
    # define the base models
    level0 = list()
    level0.append(('lr', LinearRegression()))
    level0.append(('knn', KNeighborsRegressor()))
    level0.append(('cart', DecisionTreeRegressor()))
    level0.append(('rf', RandomForestRegressor()))
    level0.append(('svm', SVR()))
    #meta learner model
    level1 = LinearRegression()
    #stacking ensemble
    model = StackingRegressor(estimators = level0, final_estimator = level1, cv =5)
    return model

def get_models():
    models = dict()
    models['lr'] = LinearRegression()
    models['knn'] = KNeighborsRegressor()
    models['cart'] = DecisionTreeRegressor()
    models['rf'] = RandomForestRegressor()
    models['svm'] = SVR()
    models['stacking'] = get_stacking()
    return models
models = get_models()

In [16]:
#create a function to iterate through different models
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

def evaluate_model(model, X, y):
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1, error_score='raise')
    return scores

In [1]:
#Turn off warnings
import warnings
warnings.filterwarnings("ignore")

#iterate through the models and append the scores
from matplotlib import pyplot
results, names = list(), list()
for name, model in models.items():
    scores = evaluate_model(model, X, y)
    results.append(scores)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))
# plot model performance for comparison
pyplot.boxplot(results, labels=names, showmeans=True)
pyplot.show()

NameError: name 'models' is not defined